In [1]:
import numpy as np
import scipy
import scipy.sparse as sp
from scipy import sparse
from scipy.sparse import eye, diags, block_diag, hstack
from scipy.sparse.csgraph import reverse_cuthill_mckee
from scipy.sparse.linalg import gmres, spsolve
from scipy.sparse import eye, diags, kron, csr_matrix
# from scipy.sparse.csgraph import symmetrix_degree_order
import importlib
import time
import AllenCahn_Solver as AC_Neu

In [2]:
importlib.reload(AC_Neu)

<module 'AllenCahn_Solver' from 'D:\\Anaconda\\Code\\EnSF\\AllenCahn\\AllenCahn_Solver.py'>

In [17]:
xa = -0.5 
xb = 0.5
ya = -0.5
yb = 0.5 
T = 10
Nt = 1000

# T = 20
# Nt = 2000

# T = 50
# Nt = 5000

# T = 100
# Nt = 10000

eps = 0.01
Nx = 128
Ny = Nx

N = Nx

hx = (xb - xa) / Nx
hy = (yb - ya) / Ny

h = hx
x = np.arange(xa, xb + hx/2, hx)  # adding hx/2 ensures xb is included
y = np.arange(ya, yb + hy/2, hy)

xmid = (x[:-1]+x[1:])/2
ymid = (y[:-1]+y[1:])/2

dt = T / Nt
TT = np.arange(0, T + dt/2, dt)

# S = 2
S = 2
cons = 0

gamma = 1
## for uniform time step
b0 = 3/(2*dt)
b1 = -1/(2*dt)

U0 = np.random.uniform(-0.9, 0.9, size=Nx*Ny)

In [18]:
# scipy.io.savemat('RefU0_AllenC_DegeneMob_v2.mat', {'U0': U0})

In [11]:
data = scipy.io.loadmat('RefU0_AllenC_DegeneMob_v1.mat')

U0 = data['U0'].astype(np.float64)
U0 = U0.reshape(Nx*Ny, )

In [19]:
# main diagonal
Main = (-2)*np.ones(N)
Main[0] = -1
Main[-1] = -1
Main *= (1/h**2)

# super- and sub-diagonals
Up =  (1/(h**2))*np.ones(N-1)
Low = (1/(h**2))*np.ones(N-1)

# assemble
G = diags(
    diagonals=[Main, Up, Low],
    offsets=[0, 1, -1],
    shape=(N, N),
    format='csr'
)
# G = np.diag(Main)       \
#   + np.diag(Up,  k= 1) \
#   + np.diag(Low,  k=-1)

I = eye(N, format='csr')
Dh = kron(I, G, format='csr') + kron(G, I, format='csr')

# Idd = np.eye(N)
# Dh = np.kron(Idd, G)+np.kron(G, Idd)

In [20]:
noise = 2*np.random.randn(Nx*Ny) # try this first
# noise = np.random.randn(Nx*Ny)
# noise = 0.4*np.random.randn(Nx*Ny)
U1 = AC_Neu.AllenCahn_Solver_1step_BDF1(U0, N, dt, eps, gamma, S, Dh, 4, noise)

In [21]:
RefSol = np.zeros((Nx*Ny, Nt+1))

RefSol[:, 0] += U0
RefSol[:, 1] += U1

In [22]:
for ii in range(1, Nt):
    print(f'step={ii}:')
    t1 = time.time() 
#     noise = 2*np.random.randn(Nx*Ny)
#     noise = np.random.randn(Nx*Ny)
    noise = 0.4*np.random.randn(Nx*Ny)
    x = AC_Neu.AllenCahn_Solver_1step_BDF2(U1, U0, N, dt, eps, gamma, S, Dh, b0, b1, 4, noise)
    t2 = time.time()
    print(f'\t time = {t2 - t1:.4f}')
    
    U0 = U1.copy()
    U1 = x.copy()
    RefSol[:, ii+1] +=x

step=1:
	 time = 0.2626
step=2:
	 time = 0.3579
step=3:
	 time = 0.2677
step=4:
	 time = 0.3707
step=5:
	 time = 0.3324
step=6:
	 time = 0.1965
step=7:
	 time = 0.5767
step=8:
	 time = 0.7262
step=9:
	 time = 0.6521
step=10:
	 time = 0.4663
step=11:
	 time = 0.7831
step=12:
	 time = 0.4731
step=13:
	 time = 0.4672
step=14:
	 time = 0.4137
step=15:
	 time = 0.5820
step=16:
	 time = 0.6285
step=17:
	 time = 0.1981
step=18:
	 time = 0.5040
step=19:
	 time = 0.4412
step=20:
	 time = 0.4356
step=21:
	 time = 0.5231
step=22:
	 time = 0.5343
step=23:
	 time = 0.2271
step=24:
	 time = 0.4956
step=25:
	 time = 0.4839
step=26:
	 time = 0.3592
step=27:
	 time = 0.3978
step=28:
	 time = 0.5683
step=29:
	 time = 0.3213
step=30:
	 time = 0.2718
step=31:
	 time = 0.6209
step=32:
	 time = 0.4576
step=33:
	 time = 0.4595
step=34:
	 time = 0.3871
step=35:
	 time = 0.6096
step=36:
	 time = 0.3503
step=37:
	 time = 0.6716
step=38:
	 time = 0.6413
step=39:
	 time = 0.5046
step=40:
	 time = 0.6623
step=41:


	 time = 0.3650
step=321:
	 time = 0.4221
step=322:
	 time = 0.3428
step=323:
	 time = 0.2707
step=324:
	 time = 0.2359
step=325:
	 time = 0.5149
step=326:
	 time = 0.3569
step=327:
	 time = 0.4466
step=328:
	 time = 0.2278
step=329:
	 time = 0.4443
step=330:
	 time = 0.3596
step=331:
	 time = 0.5256
step=332:
	 time = 0.3566
step=333:
	 time = 0.2393
step=334:
	 time = 0.5178
step=335:
	 time = 0.3970
step=336:
	 time = 0.3828
step=337:
	 time = 0.4148
step=338:
	 time = 0.4315
step=339:
	 time = 0.3267
step=340:
	 time = 0.3068
step=341:
	 time = 0.4586
step=342:
	 time = 0.5363
step=343:
	 time = 0.3063
step=344:
	 time = 0.4565
step=345:
	 time = 0.3287
step=346:
	 time = 0.4321
step=347:
	 time = 0.3098
step=348:
	 time = 0.2992
step=349:
	 time = 0.3012
step=350:
	 time = 0.3241
step=351:
	 time = 0.4981
step=352:
	 time = 0.3481
step=353:
	 time = 0.4508
step=354:
	 time = 0.7176
step=355:
	 time = 0.2695
step=356:
	 time = 0.4348
step=357:
	 time = 0.2653
step=358:
	 time = 0.3

	 time = 0.2315
step=637:
	 time = 0.2676
step=638:
	 time = 0.2732
step=639:
	 time = 0.2335
step=640:
	 time = 0.2449
step=641:
	 time = 0.2117
step=642:
	 time = 0.2280
step=643:
	 time = 0.1581
step=644:
	 time = 0.2721
step=645:
	 time = 0.2089
step=646:
	 time = 0.2676
step=647:
	 time = 0.2908
step=648:
	 time = 0.2657
step=649:
	 time = 0.2165
step=650:
	 time = 0.3076
step=651:
	 time = 0.2175
step=652:
	 time = 0.2248
step=653:
	 time = 0.2296
step=654:
	 time = 0.1886
step=655:
	 time = 0.2641
step=656:
	 time = 0.2416
step=657:
	 time = 0.2172
step=658:
	 time = 0.1810
step=659:
	 time = 0.2601
step=660:
	 time = 0.2190
step=661:
	 time = 0.2181
step=662:
	 time = 0.1766
step=663:
	 time = 0.1481
step=664:
	 time = 0.2522
step=665:
	 time = 0.1926
step=666:
	 time = 0.4050
step=667:
	 time = 0.2823
step=668:
	 time = 0.1797
step=669:
	 time = 0.1891
step=670:
	 time = 0.1781
step=671:
	 time = 0.2837
step=672:
	 time = 0.2487
step=673:
	 time = 0.2737
step=674:
	 time = 0.2

	 time = 0.1416
step=954:
	 time = 0.1610
step=955:
	 time = 0.1306
step=956:
	 time = 0.1497
step=957:
	 time = 0.1505
step=958:
	 time = 0.1186
step=959:
	 time = 0.1172
step=960:
	 time = 0.1392
step=961:
	 time = 0.1627
step=962:
	 time = 0.1241
step=963:
	 time = 0.1456
step=964:
	 time = 0.1071
step=965:
	 time = 0.1318
step=966:
	 time = 0.1596
step=967:
	 time = 0.1511
step=968:
	 time = 0.1641
step=969:
	 time = 0.1385
step=970:
	 time = 0.1343
step=971:
	 time = 0.1620
step=972:
	 time = 0.1325
step=973:
	 time = 0.1626
step=974:
	 time = 0.1375
step=975:
	 time = 0.1555
step=976:
	 time = 0.1990
step=977:
	 time = 0.1445
step=978:
	 time = 0.1352
step=979:
	 time = 0.1486
step=980:
	 time = 0.1071
step=981:
	 time = 0.1618
step=982:
	 time = 0.1245
step=983:
	 time = 0.1716
step=984:
	 time = 0.1085
step=985:
	 time = 0.0670
step=986:
	 time = 0.1652
step=987:
	 time = 0.2110
step=988:
	 time = 0.1602
step=989:
	 time = 0.1335
step=990:
	 time = 0.1363
step=991:
	 time = 0.1

In [23]:
scipy.io.savemat('RefSol_AllenC_T10_DegenerateMob_max0_v1.mat', {'RefU':RefSol}) # S=2, 0.4N(0, I)